In [1]:
import sys

sys.path.append("../../")

from app.common.logger import logger
from app.config.utils import Configuration, init_config
from app.common.llm_clients.openai_client import OpenAILLMClient

from ml.data_processing.prompt import STEP_1_3_QUESTION_GENERATION_PROMPT

In [2]:
config = Configuration()
init_config(config)
config = config()

# 데이터 불러오기

In [3]:
import json

In [ ]:
with open("../data/amr_guide_textbook_merged.json", "r", encoding="utf-8") as f:
    arm_guide_textbook = json.load(f)

In [10]:
with open("../data/step1/acs_alarm_preprocessed.json", "r", encoding="utf-8") as f:
    acs_alarm_preprocessed = json.load(f)

len(acs_alarm_preprocessed)

128

In [5]:
with open("../data/step1/mcs_textbook_merged2.json", "r", encoding="utf-8") as f:
    mcs_textbook = json.load(f)
with open("../data/step1/hacs_textbook_merged2.json", "r", encoding="utf-8") as f:
    hacs_textbook = json.load(f)

In [6]:
len(mcs_textbook), len(hacs_textbook)

(20, 76)

# 질문 생성

In [8]:
model = "google/gemini-3-flash-preview"
provider = "openrouter"
api_key = config["openrouter"]["api_key"]
params = {
    "max_tokens": 4096,
    "temperature": 0.0
}
llm_client = OpenAILLMClient(model=model, provider=provider, api_key=api_key, params=params)

2026-01-14 15:00:46.449 | INFO     | app.common.llm_clients.openai_client:_create_clients:57 - Creating OpenRouter client.


In [8]:
import re

def parse_json(text: str):
    """JSON 문자열을 파싱합니다."""
    match = re.search(r"```json\s*([\s\S]*?)\s*```", text)
    json_string = match.group(1).strip() if match else text
    return json.loads(json_string)

In [14]:
for x in arm_guide_textbook:
    try:
        questions = x['questions']
        logger.info('already have questions')
    except:
        textbook = x['textbook']
        user_prompt = f"""[Context]
    {textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x['questions'] = paresed

2026-01-05 15:31:35.227 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.228 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.228 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.231 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31

In [15]:
with open("amr_guide_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(arm_guide_textbook, f, ensure_ascii=False, indent=4)

In [9]:
for x3 in mcs_textbook:
    try:
        questions = x3['questions']
        logger.info('already have questions')
    except:
        textbook = x3['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x3['questions'] = paresed

with open("../data/mcs_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(mcs_textbook, f, ensure_ascii=False, indent=4)

2026-01-12 11:54:27.186 | INFO     | __main__:<module>:13 - ["MCS(Material Control System)를 통한 'PLC 센서 값 갱신(Update PLC Sensor Value) API'의 명세와 활용 시나리오 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", 'API를 통해 주입된 데이터가 PLC 레지스터에 반영되는 과정과, 이것이 실제 공정 로직 및 AMR 제어 시스템 내에서 어떻게 상호작용하는지 그 흐름을 공학적으로 분석하시오.', '센서 고장 또는 물리적 오차 발생 시, 수동 오버라이드(Manual Override)를 통해 공정 인터락(Interlock)을 우회하는 논리적 판단 과정과 시스템의 단계별 대응 메커니즘을 서술하시오.']
2026-01-12 11:54:30.185 | INFO     | __main__:<module>:13 - ["MCS 백엔드 서버의 가용성 진단을 위한 '서버 연결 상태 확인(Server Connectivity Check)' API의 전반적인 사양과 운영 목적에 대해 기술 백서 형식으로 상세히 서술하시오.", '시스템 초기화 및 네트워크 트러블슈팅 과정에서 /api/Object/Hello 인터페이스가 클라이언트와 서버 간의 통신 무결성을 검증하는 작동 원리와 시스템적 상호작용을 공학적으로 분석하시오.', '로봇 제어 명령 실패 또는 타임아웃 발생 시, 해당 API를 통한 네트워크 진단 결과가 시스템의 문제 원인 판별 및 후속 대응 로직에 미치는 논리적 판단 과정과 대응 메커니즘을 서술하시오.']
2026-01-12 11:54:33.088 | INFO     | __main__:<module>:13 - ["MCS(Material Control System) 내 등록된 모든 장소의 상태 정보를 조회하는 'Get All Places' API의 기술 명세와 시스템 활용 방안에 대해 기술 백서 형식으로 상세히 서술하시오.", '전체 장소 상태 조회 API가 반환하는 Al

In [10]:
for x4 in hacs_textbook:
    try:
        questions = x4['questions']
        logger.info('already have questions')
    except:
        textbook = x4['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x4['questions'] = paresed

with open("../data/hacs_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(hacs_textbook, f, ensure_ascii=False, indent=4)

2026-01-12 11:55:27.798 | INFO     | __main__:<module>:13 - ['현대 자동화 제어 시스템(HACS) WebBackend의 사용자 등록(User Registration) API 명세 및 운용 체계 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', 'BMA 공정 모니터링 권한 획득을 위한 사용자 정보 객체(userInfo)의 데이터 구조와 WebBackend 시스템 간의 자격 증명 등록 및 상호작용을 공학적으로 분석하시오.', '신규 운영자 등록 과정에서 중복 ID 발생이나 서버 내부 오류 등 예외 상황 발생 시 시스템의 논리적 판단 과정과 데이터 무결성 유지를 위한 대응 메커니즘을 서술하시오.']
2026-01-12 11:55:30.821 | INFO     | __main__:<module>:13 - ["현대 자동화 제어 시스템(HACS)의 사용자 인증 및 세션 초기화를 위한 '사용자 로그인(User Login) API' 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", '사용자 자격 증명 검증부터 JWT 발급 및 권한(Role) 부여에 이르는 인증 프로세스의 작동 원리와, 발급된 토큰이 전체 제어 시스템 내에서 API 호출 시 어떻게 상호작용하는지 공학적으로 분석하시오.', '인증 실패(401), 계정 잠금(403), 정책 거부(406) 등 예외 상황 발생 시 시스템의 논리적 판단 과정과 보안 및 운영 측면에서의 대응 메커니즘을 서술하시오.']
2026-01-12 11:55:33.544 | INFO     | __main__:<module>:13 - ["BMA 공정 제어 시스템의 '사용자 로그아웃(User Logout) API' 명세와 보안 관리 체계 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", '로그아웃 요청 시 전달되는 Bearer 토큰 기반의 인증 방식이 서버 측 세션 무효화 및 권한 회수 과정에서 어떻게 상호작용하는지 공학적으로 분석하시오.', '보안 타임아웃 또는 작업자 교대 상황 

# 데이터 포맷팅 

In [11]:
system_content = "당신은 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다."

In [12]:
think_templates = {
    0: """<think>
사용자가 '{topic}'에 대한 기술 백서 형식의 포괄적인 서술을 요청했다.
이에 따라 BMA 지식 베이스에서 해당 주제를 다루는 전체 챕터를 인출하여, 시스템 아키텍처와 엔지니어링 세부 사항을 상세히 설명한다.
</think>""",

    1: """<think>
사용자가 '{topic}'의 구체적인 공학적 작동 원리와 메커니즘에 대해 질문했다. ({question})
이 기술 요소는 단독으로 작동하지 않고 주변 시스템(PLC, 센서 등)과 유기적으로 연동되므로, 정확한 답변을 위해 전체 기술 맥락이 포함된 상세 백서를 제시하여 상호작용 흐름을 설명한다.
</think>""",

    2: """<think>
사용자가 '{topic}' 관련 상황에서의 논리적 인과관계와 시스템 대응 방식을 문의했다. ({question})
단순한 에러 해결책 제시보다 근본적인 발생 원인 분석과 Safety Interlock 로직을 이해하는 것이 중요하므로, 관련 프로토콜이 기술된 전문 분석 문서를 인출하여 답변한다.
</think>"""
}

In [28]:
result = []
for i, json_result in enumerate(arm_guide_textbook):
    if i < 7:
        text = json_result['preprocessed_group']
        textbook = json_result['textbook']
        chapter_title = text.split('\n')[0].strip().replace('#', '').strip()
        logger.info(chapter_title)
    else:
        textbook = json_result['textbook']
        chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
        logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result.append(conversation)

2026-01-05 15:51:32.545 | INFO     | __main__:<module>:7 - 1. Daily Operating
2026-01-05 15:51:32.546 | INFO     | __main__:<module>:7 - 2. Trouble Shooting: Call Assignment
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 3. Trouble Shooting: Navigation & Obstacle
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 4. Trouble Shooting: Docking & H/W
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 5. Trouble Shooting: Model Specific
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:7 - 6. Operational Manual: Control Tools (조작 및 유지보수 매뉴얼)
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:7 - 7. Appendix
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:11 - WEB ACS 기반의 Place Cycle 제어 논리와 미션 디스패칭(Mission Dispatching) 메커니즘
2026-01-05 15:51:32.549 | INFO     | __main__:<module>:11 - WEB ACS 기반의 동적 물류 제어 메커니즘: Place Cycle 및 가동 상태 동기화 프로토콜
2026-01-05 15:51:32.549 | INFO     | __main__:<module>:11 - BMA 공정 내 자율주행 로봇(AMR)의 전력 관리 및 운영 생명주기 프로토콜 (Energy

In [29]:
len(result)

192

In [30]:
with open("arm_guide_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [24]:
import re

def parse_log_line(text):
    """
    Markdown 테이블 형식의 문자열(여러 줄 포함)에서 데이터를 추출합니다.
    입력 text 예시:
    | Part | No. | Contents (Kor) | ...
    | :--- | :-- | :------------- | ...
    | 테스크 | 10000 | RFID 감지 안됨 | [10000] RFID not detected | ...
    """
    # 1. 텍스트를 줄 단위로 분리 (헤더, 구분선, 데이터 라인을 구분하기 위함)
    lines = text.strip().split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 2. 파이프(|)를 기준으로 분리 및 앞뒤 공백 제거
        parts = [p.strip() for p in line.split('|')]

        # 유효성 검사:
        # - 분리된 요소가 최소 6개 이상이어야 함 (양끝 빈 문자열 포함)
        # - 헤더('No.')나 구분선('---')이 포함된 라인은 건너뜀
        if len(parts) < 6:
            continue
        if 'No.' in parts[2] or '---' in parts[2]:
            continue

        # 3. 데이터 추출 (테이블 구조에 따른 인덱스)
        # parts[2]: No. (예: 10000)
        # parts[3]: Contents (Kor) (예: RFID 감지 안됨)
        # parts[4]: Contents (Eng) (예: [10000] RFID not detected)
        # parts[5]: LEVEL (예: Warn)

        no_val = parts[2]
        kor_text = parts[3]
        raw_eng_text = parts[4]
        level = parts[5]

        # 4. 영문 텍스트 전처리: [숫자] 제거
        # 정규식 설명:
        # ^ : 문자열 시작
        # \\? : 백슬래시(\)가 0개 또는 1개 (JSON 이스케이프 대응)
        # \[ : 여는 대괄호
        # \d+ : 숫자 1개 이상
        # \\? : 백슬래시(\)가 0개 또는 1개
        # \] : 닫는 대괄호
        # \s* : 뒤따르는 공백
        eng_text = re.sub(r'^\\?\[\d+\\?\]\s*', '', raw_eng_text)

        # 유효한 데이터 행을 찾으면 즉시 반환
        return {
            "no": no_val,
            "content_kor": kor_text.replace("\\", ""),
            "content_eng": eng_text.replace("\\", ""),
            "level": level
        }

    return None


system_content = "당신은 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다."

user_question_think_templates = ["'[{code}] {name}' 알람은 무엇입니까?", "{code} 알람이 발생했어. 뭐야?"]
sop_think_templates = """<think>
사용자가 '[{code}] {name}' 상황에 대한 설명을 요청했다.
이에 따라 해당 알람에 대한 간단한 설명을 제시한다.
</think>"""

In [25]:
result2 = []
for i, json_result in enumerate(acs_alarm_preprocessed):
    original = json_result['original']
    original_data = parse_log_line(original)
    code = original_data['no']
    name = original_data['content_kor']
    text = json_result['preprocessed']


    for question in user_question_think_templates:
        q = question.format(code=code, name=name)
        think_content = sop_think_templates.format(code=code, name=name)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + text
                }
            ]
        }
        print(conversation)
        result2.append(conversation)

{'conversation': [{'role': 'system', 'content': '당신은 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다.'}, {'role': 'user', 'content': "'[10000] RFID 감지 안됨' 알람은 무엇입니까?"}, {'role': 'assistant', 'content': "<think>\n사용자가 '[10000] RFID 감지 안됨' 상황에 대한 설명을 요청했다.\n이에 따라 해당 알람에 대한 간단한 설명을 제시한다.\n</think>\n\nBMA 공정 AMR의 RFID 리더기가 태그 정보를 정상적으로 인식하지 못할 때 10000번 경고(RFID not detected)가 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 작업자는 AMR의 정지 위치를 조정하여 RFID 태그와 리더기 사이의 통신 거리를 확보하거나 태그의 부착 상태를 점검해야 한다."}]}
{'conversation': [{'role': 'system', 'content': '당신은 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다.'}, {'role': 'user', 'content': '10000 알람이 발생했어. 뭐야?'}, {'role': 'assistant', 'content': "<think>\n사용자가 '[10000] RFID 감지 안됨' 상황에 대한 설명을 요청했다.\n이에 따라 해당 알람에 대한 간단한 설명을 제시한다.\n</think>\n\nBMA 공정 AMR의 RFID 리더기가 태그 정보를 정상적으로 인식하지 못할 때 10000번 경고(RFID not detected)가 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 작업자는 AMR의 정지 위치를 조정하여 RFID 태그와 리더기 사이의

In [26]:
len(result2)

256

In [27]:
with open("acs_alarm_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result2, f, ensure_ascii=False, indent=4)

In [13]:
result3 = []
for i, json_result in enumerate(mcs_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('##', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result3.append(conversation)

logger.info(len(result3))

with open("../data/mcs_textbook_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result3, f, ensure_ascii=False, indent=4)

2026-01-12 11:59:13.442 | INFO     | __main__:<module>:5 - [API] PLC 센서 값 갱신 (Update PLC Sensor Value)
2026-01-12 11:59:13.443 | INFO     | __main__:<module>:5 - [API] 서버 연결 상태 확인 (Server Connectivity Check)
2026-01-12 11:59:13.444 | INFO     | __main__:<module>:5 - [API] 전체 장소 상태 조회 (Get All Places)
2026-01-12 11:59:13.445 | INFO     | __main__:<module>:5 - [API] 장소 상태 정보 갱신 (Update Place Status)
2026-01-12 11:59:13.445 | INFO     | __main__:<module>:5 - [API] 전체 호출 정보 조회 (Retrieve All Calls)
2026-01-12 11:59:13.445 | INFO     | __main__:<module>:5 - [API] 전체 미션 목록 조회 (Retrieve All Missions)
2026-01-12 11:59:13.446 | INFO     | __main__:<module>:5 - [API] 호출 장소 상태 조회 (Get Call Places Status)
2026-01-12 11:59:13.446 | INFO     | __main__:<module>:5 - [API] MCS 알람 정보 조회 (Retrieve MCS Alarms)
2026-01-12 11:59:13.446 | INFO     | __main__:<module>:5 - [API] AMR 이동 명령 (Move Command)
2026-01-12 11:59:13.447 | INFO     | __main__:<module>:5 - [API] 호출 상태 변경 (Change Call State)
2026-01-12 11:

In [14]:
result4 = []
for i, json_result in enumerate(hacs_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('##', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result4.append(conversation)

logger.info(len(result4))
with open("../data/hacs_textbook_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result4, f, ensure_ascii=False, indent=4)

2026-01-12 11:59:13.458 | INFO     | __main__:<module>:5 - [API] 사용자 등록 (User Registration)
2026-01-12 11:59:13.459 | INFO     | __main__:<module>:5 - [API] 사용자 로그인 (User Login)
2026-01-12 11:59:13.459 | INFO     | __main__:<module>:5 - [API] 사용자 로그아웃 (User Logout)
2026-01-12 11:59:13.460 | INFO     | __main__:<module>:5 - [API] 실행중인 작업 조회 (Get Running Work Orders)
2026-01-12 11:59:13.460 | INFO     | __main__:<module>:5 - [API] 작업 이력 조회 (Work Order History)
2026-01-12 11:59:13.460 | INFO     | __main__:<module>:5 - [API] 작업 오더 삭제 (Delete Work Order)
2026-01-12 11:59:13.461 | INFO     | __main__:<module>:5 - [API] 객체 제어 모듈 연결 테스트 (Object Controller Connectivity Test)
2026-01-12 11:59:13.461 | INFO     | __main__:<module>:5 - [API] 서비스 가용성 진단 (Service Availability Check / Hello)
2026-01-12 11:59:13.461 | INFO     | __main__:<module>:5 - [API] 맵 객체 전체 조회 (Retrieve Map Objects)
2026-01-12 11:59:13.462 | INFO     | __main__:<module>:5 - [API] 맵 스타일 구성 정보 조회 (Get Map Style Configuration)
20